<a href="https://colab.research.google.com/github/Minjule/registration-sys/blob/main/vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from zipfile import ZipFile
file_path = "/content/covid19-image-dataset.zip"
with ZipFile(file_path, 'r') as zip_file:
  zip_file.extractall()
  print('Extraction Compelted')

In [ ]:
import os
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random as python_random
import tensorflow as tf
import seaborn as sns
import math
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.callbacks import ModelCheckpoint
from  tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
train_dir = "/content/Covid19-dataset/train"
test_dir = "/content/Covid19-dataset/test"

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=True,
    classes={'Normal': 0, 'Viral Pneumonia': 1,'Covid': 2}
    )

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=False,
    classes={'Normal': 0, 'Viral Pneumonia': 1,'Covid': 2}
    )

# shuffle=True will shuffle the images and labels within each batch
#  but the correspondence between the images and labels will be maintained.

In [ ]:
total_image = np.concatenate([train_generator.labels,test_generator.labels])

In [ ]:
print('Normal_cases: ',len(np.where(total_image==0)[0]))
print('Viral_Pneumonia_cases: ',len(np.where(total_image==1)[0]))
print('Covid_cases: ',len(np.where(total_image==2)[0]))

# ***Implementing VGG16 - Transfer Learning***

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
# Loading VGG16 Model with Pre-trained Weights - without final (top) later as we will customize it
base_model = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(HEIGHT, WIDTH, 3))

In [ ]:
# Freeze the layers of the base model
# so that ony final layer learns from new dataset
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
num_classes = len(train_generator.class_indices)
print('Number of classes:', num_classes)

In [ ]:
# 3. Fine-tune the Model
model = Sequential()
model.add(base_model)
# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))

model.add(GlobalAveragePooling2D())

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

#
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, to_file='vgg16-image-classifier.png', show_shapes=True, show_layer_names=True)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='SparseCategoricalCrossentropy',
              metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint('vgg16_best_weights.h5',
                             monitor='val_accuracy',
                            #  monitor='val_f1_score',
                             verbose=1,
                             mode='max',
                             save_best_only=True)

early = EarlyStopping(monitor="val_loss",
                      mode="min",
                      restore_best_weights=True,
                      patience=5)

callbacks_list = [checkpoint,early]

In [ ]:
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
    callbacks=[callbacks_list],
    verbose=True,
    shuffle=True)

In [ ]:
def plot_learning_curve(history, metrics):
    acc = history.history[metrics]
    val_acc = history.history[f'val_{metrics}']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.figure(figsize=(5,3))
    plt.plot(epochs, acc, label=f'training {metrics}')
    plt.plot(epochs, val_acc, label=f'validation {metrics}')
    plt.grid()
    plt.xlabel('Epochs')
    plt.ylabel(metrics)
    plt.legend()

    plt.figure(figsize=(5,3))
    plt.plot(epochs, loss, label='training loss')
    plt.plot(epochs, val_loss, label='validation loss')
    plt.grid()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

In [ ]:
train_result = model.evaluate(train_generator)
test_result = model.evaluate(test_generator)

In [ ]:
print(f"Train Loss: {np.round(train_result[0], 4)}")
print(f"Test Loss: {np.round(test_result[0], 4)}")
print(f"Train Accuracy: {np.round(train_result[1], 3)}")
print(f"Test Accuracy: {np.round(test_result[1], 3)}")

In [ ]:
y_pred_probs = model.predict(test_generator)
print(y_pred_probs.shape)

In [ ]:
y_true = test_generator.classes
y_true

In [ ]:
y_true.shape

In [ ]:
print('Normal_cases: ',len(np.where(y_true==0)[0]))
print('Viral_Pneumonia_cases: ',len(np.where(y_true==1)[0]))
print('Covid_cases: ',len(np.where(y_true==2)[0]))

In [ ]:
y_pred_probs = model.predict(test_generator)

In [ ]:
classes_names = list(test_generator.class_indices.keys())

In [ ]:
# Convert predicted probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)
# Calculate confusion matrix
confusion = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", confusion)
# Calculate perfect confusion matrix
p_confusion = confusion_matrix(y_true, y_true)
print("Perfect Confusion Matrix:\n", p_confusion)
# Calculate precision, recall, and f1-score
report = classification_report(y_true, y_pred, target_names=classes_names)
print("\nClassification Report:\n", report)

In [ ]:
plt.figure(figsize=(5, 5))
hmap = sns.heatmap(confusion, annot=True, vmin=0,
                 fmt='g', cmap='Blues', cbar=False,
                 xticklabels=classes_names,
                 yticklabels=classes_names)

hmap.set(xlabel='Predicted Labels')
hmap.set(ylabel='True Labels')
plt.show()

In [ ]:
#Extract wrong classification index
wrong_pred = np.where(y_pred!=y_true)[0]
print(f'Total of {len(wrong_pred)} missclassified images')
print('Missclassified images index: ',wrong_pred)

In [ ]:
class_map = {0: 'Normal', 1: 'Viral Pneumonia', 2: 'Covid'}

plt.figure(figsize=(10, 5))
for i, index in enumerate(wrong_pred):
    plt.subplot(1, len(wrong_pred), i + 1)

    # reverting scalling (done in preprocessing stage)
    plt.imshow((images[index]*255).astype('uint8'))

    plt.text(5, 10, f"True: {class_map[int(true_labels[index])]}",
             color='green', bbox=dict(facecolor='white', alpha=0.6))
    plt.text(5, 30, f"Pred: {class_map[y_pred[index]]}",
             color='red', bbox=dict(facecolor='white', alpha=0.6))

    plt.axis('off')
plt.tight_layout()
plt.show()